In [1]:
import torch
device = 'cuda' if torch.cuda.is_available else 'cpu'
from agent import Agent, advanced_random_policy, random_policy, sarsa
from game import Game
import matplotlib.pyplot as plt
import numpy as np

### In a first time, we train an agent for 2D tic-tac-toe using our modified SARSA algorithm

In [2]:
%%time
game = Game(None, None, n_dim=2, size=3)
agent = Agent(size=3)

n_eps = 20000
# Trains agent with the advanced policy
sarsa(game, agent, advanced_random_policy, alpha=0.45, alpha_factor=0.9995**(10000/n_eps), gamma=0.7, epsilon=1.0, \
      epsilon_factor=0.9997**(10000/n_eps), r_win=11.0, r_lose=0.0, r_even=1.0, r_even2=1.25, num_episodes=20000)

100%|██████████| 20000/20000 [02:22<00:00, 140.09it/s]

CPU times: user 2min 23s, sys: 6.24 s, total: 2min 29s
Wall time: 2min 22s


We make our agent play 1000 games against the random policy

In [3]:
random_opponent = Agent(size=3, policy=random_policy)
game = Game(agent, random_opponent, n_dim=2, size=3)

agent_1_win, oppo_2_win, agent_1_even, agent_2_win, oppo_1_win, agent_2_even = game.simulate_games(1000)
tot_agent_win = agent_1_win + agent_2_win
tot_agent_lose = oppo_1_win + oppo_2_win
tot_even = agent_1_even + agent_2_even
print("On 1000 games, agent won", tot_agent_win, "times, lost", tot_agent_lose, "times and made", tot_even, "even games.")

On 1000 games, agent won 895 times, lost 0 times and made 105 even games.


Now we make our agent play 1000 games against the advanced policy

In [4]:
advanced_opponent = Agent(size=3, policy=advanced_random_policy)
game = Game(agent, advanced_opponent, n_dim=2, size=3)

agent_1_win, oppo_2_win, agent_1_even, agent_2_win, oppo_1_win, agent_2_even = game.simulate_games(1000)
tot_agent_win = agent_1_win + agent_2_win
tot_agent_lose = oppo_1_win + oppo_2_win
tot_even = agent_1_even + agent_2_even
print("On 1000 games, agent won", tot_agent_win, "times, lost", tot_agent_lose, "times and made", tot_even, "even games.")

On 1000 games, agent won 253 times, lost 0 times and made 747 even games.


We give the possibility to play against our agent

In [5]:
game = Game(agent, "Human player", n_dim=2, size=3) # agent plays first
#game = Game("Human player", agent, n_dim=2, size=3) # to play first
game.play_a_game()

. . . 
. . . 
. . . 

Agent plays : (1, 1) 

. . . 
. X . 
. . . 

Coordinates of next move : 0 0

O . . 
. X . 
. . . 

Agent plays : (0, 1) 

O X . 
. X . 
. . . 

Coordinates of next move : 2 1

O X . 
. X . 
. O . 

Agent plays : (1, 2) 

O X . 
. X X 
. O . 

Coordinates of next move : 1 0

O X . 
O X X 
. O . 

Agent plays : (2, 0) 

O X . 
O X X 
X O . 

Coordinates of next move : 0 2

O X O 
O X X 
X O . 

Agent plays : (2, 2) 

O X O 
O X X 
X O X 

Game over. Score : (0, 0)
Even score.


(0, 0)

We can evaluate the performances of our agent depending on the number of training episodes

In [ ]:
win1, lose1, draw1, win2, lose2, draw2, n_eps = [], [], [], [], [], [], []
for n in [1, 10, 100, 200, 400, 700, 1000, 2000, 3000, 4000, 5000, 7000, 10000, 15000, 20000]:
    ar = 0.9995**(10000/n)
    er = 0.9997**(10000/n)
    agent = Agent(size=3)
    advanced_opponent = Agent(size=3, policy=advanced_random_policy)
    game = Game(agent, advanced_opponent, n_dim=2, size=3)
    sarsa(game, agent, random_policy, alpha=0.45, alpha_factor=ar, gamma=0.7, epsilon=1.0, epsilon_factor=er, \
          r_win=11.0, r_lose=0.0, r_even=1.0, r_even2=1.25, num_episodes=n)
    win_p1_a, win_p2_a, tot_even_a, win_p1_b, win_p2_b, tot_even_b = game.simulate_games(10000)
    win1.append(win_p1_a)
    win2.append(win_p1_b)
    lose1.append(win_p2_a)
    lose2.append(win_p2_b)
    draw1.append(tot_even_a)
    draw2.append(tot_even_b)
    n_eps.append(n)

Results when the agent plays in first

In [ ]:
n_eps = np.array(n_eps)
fig = plt.figure(figsize=(0.6*6.4, 1*4.8))
plt.plot(n_eps, np.array(win1)/50, label='win')
plt.plot(n_eps, np.array(lose1)/50, label='lose')
plt.plot(n_eps, np.array(draw1)/50, label='draw')
plt.xscale('log')
plt.xlabel('number of episodes played in training', fontsize='large')
plt.ylabel('test games (in % on 5000 games)', fontsize='large')
plt.ylim(0, 105)
plt.title('Fig.(7) : Training with the random\npolicy and testing against the\nadvanced one (agent plays in first)')
plt.legend(loc='best', fontsize='large')
plt.savefig("plot1.pdf", bbox_inches='tight')

Results when the agent plays in second

In [ ]:
n_eps = np.array(n_eps)
fig = plt.figure(figsize=(0.6*6.4, 1*4.8))
plt.plot(n_eps, np.array(win2)/50, label='win')
plt.plot(n_eps, np.array(lose2)/50, label='lose')
plt.plot(n_eps, np.array(draw2)/50, label='draw')
plt.xscale('log')
plt.xlabel('number of episodes played in training', fontsize='large')
plt.ylabel('test games (in % on 5000 games)', fontsize='large')
plt.ylim(0, 105)
plt.title('Fig.(8) : Training with the random\npolicy and testing against the\nadvanced one (agent plays in second)')
plt.legend(loc='best', fontsize='large')
plt.savefig("plot2.pdf", bbox_inches='tight')

### Now, we show our Policy Gradient model for 2D, 3D and 4D
We start by the 2D case

We train our model with 20 000 iteration and batch size of 1000

In [14]:
import neuralAgent as na
device = 'cuda' if torch.cuda.is_available else 'cpu'

game = Game(None, None, n_dim=2, size=3)
agent1 = na.Model()
agent1, values, _, _, _ = na.train_network(agent1, game, 20000, 1000)
agent1.save()
print(values)

100%|██████████| 20000/20000 [01:32<00:00, 216.84it/s]

[array(0.1919922, dtype=float32), array(0.1919922, dtype=float32), array(0.23683336, dtype=float32), array(0.23683336, dtype=float32), array(0.24077186, dtype=float32), array(0.24077186, dtype=float32), array(0.23229553, dtype=float32), array(0.23229553, dtype=float32), array(0.21925719, dtype=float32), array(0.21925719, dtype=float32), array(0.23385824, dtype=float32), array(0.23385824, dtype=float32), array(0.2421333, dtype=float32), array(0.2421333, dtype=float32), array(0.22941184, dtype=float32), array(0.22941184, dtype=float32), array(0.23332284, dtype=float32), array(0.23332284, dtype=float32), array(0.22402705, dtype=float32), array(0.22402705, dtype=float32), array(0.22560924, dtype=float32), array(0.22560924, dtype=float32), array(0.23400809, dtype=float32), array(0.23400809, dtype=float32), array(0.22613238, dtype=float32), array(0.22613238, dtype=float32), array(0.22953445, dtype=float32), array(0.22953445, dtype=float32), array(0.22006044, dtype=float32), array(0.22006044,

We make our agent play 1000 games and then 10000 against the random policy

In [9]:
# Example of loading a saved agent
loaded_agent = na.Model()
loaded_agent.load()
wins, draw, loses = na.test_against_random(agent1, game, 1000)
print("Win {}, Draw {}, Loses {}".format(wins, draw, loses))

100%|██████████| 1000/1000 [00:04<00:00, 220.19it/s]

Win 850, Draw 38, Loses 112


In [10]:
wins, draw, loses = na.test_against_random(agent1, game, 10000)
print("Win {}, Draw {}, Loses {}".format(wins, draw, loses))

100%|██████████| 10000/10000 [00:42<00:00, 233.73it/s]

Win 8770, Draw 291, Loses 939


In [11]:
game = Game(agent1, "Human player", n_dim=2, size=3) # agent plays first
#game = Game("Human player", agent1, n_dim=2, size=3) # to play first
game.play_a_game()

. . . 
. . . 
. . . 

Agent plays : (0, 2) 

. . X 
. . . 
. . . 

Coordinates of next move : 1 1

. . X 
. O . 
. . . 

Agent plays : (1, 2) 

. . X 
. O X 
. . . 

Coordinates of next move : 2 2

. . X 
. O X 
. . O 

Agent plays : (2, 0) 

. . X 
. O X 
X . O 

Coordinates of next move : 0 0

O . X 
. O X 
X . O 

Game over. Score : (0, 1)
Human player wins !


(0, 1)

#### Now the 3D case 

In [12]:
game = Game(None, None, n_dim=3, size=3)
agent1 = na.Model(3, 3)
agent1, values, _, _, _ = na.train_network(agent1, game, 10000, 1000)
#print(values)

100%|██████████| 10000/10000 [10:23<00:00, 16.03it/s]


We show its performances against the random policy

In [13]:
wins, draw, loses = na.test_against_random(agent1, game, 1000)
print("Win {}, Draw {}, Loses {}".format(wins, draw, loses))

100%|██████████| 1000/1000 [01:01<00:00, 16.13it/s]

Win 912, Draw 39, Loses 49
